In [1]:
import numpy as np
import os
import json
import cv2 as cv
import shutil

## Set labeled imgs

In [2]:
selected_images={"GX010034_14.jpg",
                 "GX010040_19.jpg",
                 "GX010034_10.jpg",
                 "GX010035_50.jpg",
                 "GX010044_4.jpg",
                 "GX010038_1.jpg",
                 "GX010045_15.jpg",
                 "GX010046_6.jpg",
                 "GX010045_17.jpg",
                 "GX010044_6.jpg",
                 "GX010045_17.jpg",
                 "GX010044_6.jpg",
                 "GX010040_31.jpg",
                 "GX010038_17.jpg",
                 "GX010045_29.jpg",
                 "GX010044_5.jpg",
                 "GX010040_3.jpg",
                 "GX010042_1.jpg",
                 "GX010032_8.jpg"}

path_image="/Users/YaVolkonskiy/Documents/Projects/tomato/data/001_raw_data/001_video_data/002_GO_PRO_SBER_20231101_ALL/labeled_Olga_Gelya_21_11_23/img_selected/"

## Load labeled LS anot 

In [3]:
# pth_name = "project-13-at-2023-11-21-11-39-a223ddf0.json"
pth_name ="/Users/YaVolkonskiy/Documents/Projects/tomato/data/001_raw_data/001_video_data/002_GO_PRO_SBER_20231101_ALL/labeled_Olga_Gelya_21_11_23/project-13-at-2023-11-21-11-39-a223ddf0.json"

with open(pth_name) as user_file:
    parsed_json = json.load(user_file)


# Select just labeled agree with set
need_labels= []

for n in parsed_json:
    if os.path.basename(n['data']['image']) in selected_images:
        need_labels.append(n)

## Pipe convert LS to panoptic_seg

In [4]:
def relative2absalute(np_arr, resolution_orig):
    np_arr_abs = np.copy(np_arr)
    np_arr_abs[:,:1] = np_arr_abs[:,:1]/100*resolution_orig[1]
    np_arr_abs[:,1:] = np_arr_abs[:,1:]/100*resolution_orig[0]
    # np_arr_abs=np.rint(np_arr_abs).astype(np.int32)
    np_arr_abs=np.rint(np_arr_abs).astype(np.int32)

    return np_arr_abs

def get_resolution(one_imgObj_LS):
    w_orig = one_imgObj_LS["annotations"][0]['result'][0]['original_width']
    h_orig = one_imgObj_LS["annotations"][0]['result'][0]['original_height']
    resolution_orig = (h_orig,w_orig)
    return resolution_orig

def LS2panoptic_map(one_imgObj_LS, classes, init_id):
    
    resolution_orig=get_resolution(one_imgObj_LS)
    full_mask = np.zeros(shape=resolution_orig, dtype=np.uint32)
    inst2class = {}
    
    for result in one_imgObj_LS["annotations"][0]["result"]:
        points=result["value"]['points']
        np_polyg=np.array( [np.array(i) for i in points] )
        # Convert relative points to absolute
        np_polyg_abs=relative2absalute(np_polyg, resolution_orig)
        # Add dimension polyg
        np_polyg_abs=np_polyg_abs[np.newaxis, ... ] 
        # Create temporaty mask
        mask = np.zeros(shape=resolution_orig, dtype=np.uint8)
        # we can't use here number more than 255 fot this cv.fillPoly functione there fore we fill value=1, then will replace it to => init_ids
        filled_mask = cv.fillPoly(mask, pts=np_polyg_abs, color=1)

        # fill filled_mask by target id
        filled_mask=filled_mask.astype(np.uint32)
        filled_mask[filled_mask == 1] = init_id
        # join full_mask and curent filed mask
        full_mask[filled_mask == init_id] = init_id
        # fill inst2class
        name_label = result["value"]["polygonlabels"][0]
        # print(f"name_label {name_label}")
        inst2class[init_id] = classes[name_label]
        init_id+=1

    ## add background
    full_mask[full_mask==0]=init_id
    inst2class[init_id] = classes["background"]
    init_id+=1

    return full_mask, inst2class, init_id


## Pipe prepare

In [ ]:
schem panoptic_seg_gt

panoptic_seg_gt = {
                   "GX010032_8.jpg"
                           :{"panoptic_mask": np.ndarray,
                              "inst2class": dict_inst2class},
                   "GX010034_10.jpg"
                           :{"panoptic_mask": np.ndarray,
                              "inst2class": dict_inst2class},
#                   .....
#                   .....
                   "GX010034_14.jpg"
                           :{"panoptic_mask": np.ndarray,
                              "inst2class": dict_inst2class}
                  }

panoptic_mask = (
      [[  43,   43,   43, ...,   98,   98,   98],
       [  7,    43,   43, ...,   67,   98,   98],
       [  7,     7,    7, ...,   67,   98,   98],
       ...,
       [  139,   139,   139, ..., 108, 108, 108],
       [  19,     19,   139, ..., 108, 108, 108],
       [  19,     19,   139, ..., 108, 108, 108]], dtype=uint32)

panoptic_mask.shape = np.array (x,y)


inst2class = {43:  2,  7: 5,
              98:  30, 67: 5, 
              .....
              139: 17, 108: 11}

## not nescseary
id2label = {0: 'wall', 1: 'building', 2: 'sky', 3: 'floor', 4: 'tree', 5: 'ceiling', 6: 'road', 7: 'bed ', 8: 'windowpane', 
            9: 'grass', 10: 'cabinet', 11: 'sidewalk', 12: 'person', 13: 'earth', 14: 'door', 15: 'table', 16: 'mountain', 
            17: 'plant', 18: 'curtain', 19: 'chair', 20: 'car', 21: 'water', 22: 'painting', 23: 'sofa', 24: 'shelf', 25: 'house', 
            26: 'sea', 27: 'mirror', 28: 'rug', 29: 'field', 30: 'armchair', 31: 'seat', 32: 'fence', 33: 'desk', 34: 'rock', 
            35: 'wardrobe', 36: 'lamp', 37: 'bathtub', 38: 'railing', 39: 'cushion', 40: 'base', 41: 'box', 42: 'column', 
            43: 'signboard', 44: 'chest of drawers', 45: 'counter', 46: 'sand', 47: 'sink', 48: 'skyscraper', 49: 'fireplace', 
            50: 'refrigerator', 51: 'grandstand', 52: 'path', 53: 'stairs', 54: 'runway', 55: 'case', 56: 'pool table', 57: 'pillow', 
            58: 'screen door', 59: 'stairway', 60: 'river', 61: 'bridge', 62: 'bookcase', 63: 'blind', 64: 'coffee table', 65: 'toilet', 
            66: 'flower', 67: 'book', 68: 'hill', 69: 'bench', 70: 'countertop', 71: 'stove', 72: 'palm', 73: 'kitchen island', 74: 'computer', 
            75: 'swivel chair', 76: 'boat', 77: 'bar', 78: 'arcade machine', 79: 'hovel', 80: 'bus', 81: 'towel', 82: 'light', 83: 'truck', 
            84: 'tower', 85: 'chandelier', 86: 'awning', 87: 'streetlight', 88: 'booth', 89: 'television receiver', 90: 'airplane', 
            91: 'dirt track', 92: 'apparel', 93: 'pole', 94: 'land', 95: 'bannister', 96: 'escalator', 97: 'ottoman', 98: 'bottle', 
            99: 'buffet', 100: 'poster', 101: 'stage', 102: 'van', 103: 'ship', 104: 'fountain', 105: 'conveyer belt', 106: 'canopy', 
            107: 'washer', 108: 'plaything', 109: 'swimming pool', 110: 'stool', 111: 'barrel', 112: 'basket', 113: 'waterfall', 114: 'tent', 
            115: 'bag', 116: 'minibike', 117: 'cradle', 118: 'oven', 119: 'ball', 120: 'food', 121: 'step', 122: 'tank', 123: 'trade name', 
            124: 'microwave', 125: 'pot', 126: 'animal', 127: 'bicycle', 128: 'lake', 129: 'dishwasher', 130: 'screen', 131: 'blanket', 
            132: 'sculpture', 133: 'hood', 134: 'sconce', 135: 'vase', 136: 'traffic light', 137: 'tray', 138: 'ashcan', 139: 'fan', 
            140: 'pier', 141: 'crt screen', 142: 'plate', 143: 'monitor', 144: 'bulletin board', 145: 'shower', 146: 'radiator', 147: 'glass', 
            148: 'clock', 149: 'flag'}

In [ ]:
need_labels

In [5]:
panoptic_seg_gt={}
classes = { "leaf":0,
            "stem":1,
            "tomato":2,
            "background":3}

path_image="/Users/YaVolkonskiy/Documents/Projects/tomato/data/001_raw_data/001_video_data/002_GO_PRO_SBER_20231101_ALL/labeled_Olga_Gelya_21_11_23/img_selected/"
init_id = 2

for one_imgObj_LS in need_labels:
    name_image = os.path.basename(one_imgObj_LS['data']['image'])

    # img_read cv2
    full_img_path = os.path.join(path_image, name_image)
    image_np = cv.cvtColor(  cv.imread(full_img_path) , cv.COLOR_BGR2RGB).astype(np.float32)
    
    panoptic_mask, inst2class, init_id = LS2panoptic_map(one_imgObj_LS, classes, init_id)

    panoptic_seg_gt[name_image] = {"panoptic_mask" : panoptic_mask,
                                  "inst2class" : inst2class,
                                  "image":image_np}

In [6]:
init_id

2286

In [7]:
len(panoptic_seg_gt)

17

In [8]:
import pickle

path_to_save = "/Users/YaVolkonskiy/Documents/Projects/tomato/data/001_raw_data/001_video_data/002_GO_PRO_SBER_20231101_ALL/labeled_Olga_Gelya_21_11_23/panoptic_mask2former_uniqID_add_backgrd.pickle"
with open(path_to_save, 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(panoptic_seg_gt, f, pickle.HIGHEST_PROTOCOL)

### Another previos labels

In [9]:
selected_images={"GX010027_42.jpg",
                 "GX010027_1.jpg",
                 "GX010028_70.jpg",
                 "GX010020_89.jpg",
                 "GX010027_6.jpg",
                 "GX010017_20.jpg",
                 "GX010023_34.jpg",
                 "GX010027_46.jpg",
                 "GX010018_63.jpg",
                 "GX010026_12.jpg",
                 "GX010024_4.jpg",
                 "GX010025_60.jpg",
                 "GX010028_20.jpg",
                 "GX010018_9.jpg",
                 "GX010021_34.jpg", # from this point new labels
                 "GX010027_37.jpg",
                 "GX010019_6.jpg",
                 "GX010023_13.jpg",
                 "GX010020_43.jpg",
                 "GX010023_55.jpg",
                 "GX010019_85.jpg",
                 "GX010026_73.jpg",
                 "GX010027_24.jpg",
                 "GX010027_19.jpg",
                 "GX010020_10.jpg",
                 "GX010028_60.jpg",
                 "GX010027_28.jpg"
                 }

path_image="/Users/YaVolkonskiy/Documents/Projects/tomato/data/001_raw_data/001_video_data/001_sber_video/003_labels/002_labeled_studio_LEAF_TOMATO_15.11.23/images/"


In [10]:
len(selected_images)

27

In [11]:
# pth_name = "project-13-at-2023-11-21-11-39-a223ddf0.json"
pth_name ="/Users/YaVolkonskiy/Documents/Projects/tomato/data/001_raw_data/001_video_data/001_sber_video/003_labels/002_labeled_studio_LEAF_TOMATO_15.11.23/project-8-at-2023-11-15-17-09-e7686567.json"

with open(pth_name) as user_file:
    parsed_json = json.load(user_file)


# Select just labeled agree with set
need_labels= []

for n in parsed_json:
    if os.path.basename(n['data']['image']) in selected_images:
        need_labels.append(n)

In [ ]:
init_id

In [12]:
# panoptic_seg_gt={}
classes = { "leaf":0,
            "stem":1,
            "tomato":2,
            "background":3}

path_image="/Users/YaVolkonskiy/Documents/Projects/tomato/data/001_raw_data/001_video_data/001_sber_video/003_labels/002_labeled_studio_LEAF_TOMATO_15.11.23/images/"
# init_id = 2

for one_imgObj_LS in need_labels:
    name_image = os.path.basename(one_imgObj_LS['data']['image'])

    # img_read cv2
    full_img_path = os.path.join(path_image, name_image)
    image_np =  cv.cvtColor( cv.imread(full_img_path) , cv.COLOR_BGR2RGB).astype(np.float32)
    # cv.cvtColor( cv.imread(full_pth) , cv.COLOR_BGR2RGB).astype(np.float32)
    
    panoptic_mask, inst2class, init_id = LS2panoptic_map(one_imgObj_LS, classes, init_id)

    panoptic_seg_gt[name_image] = {"panoptic_mask" : panoptic_mask,
                                  "inst2class" : inst2class,
                                  "image":image_np}



In [ ]:
init_id

In [ ]:
len(panoptic_seg_gt)

In [ ]:
# # panoptic_seg_gt={}
# classes = { "leaf":0,
#             "stem":1,
#             "tomato":2}

# for one_imgObj_LS in need_labels:
#     name_image = os.path.basename(one_imgObj_LS['data']['image'])
#     panoptic_mask, inst2class = LS2panoptic_map(one_imgObj_LS, classes)

#     panoptic_seg_gt[name_image] = {"panoptic_mask" : panoptic_mask,
#                                   "inst2class" : inst2class}

In [ ]:
len(panoptic_seg_gt)

In [13]:
import pickle

path_to_save = "/Users/YaVolkonskiy/Documents/Projects/tomato/data/002_data_experiments/004_train_Mask2Former_panoptic_23_10_23/panoptic_mask2former_uinqID_RGB_backgrd.pickle"
with open(path_to_save, 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(panoptic_seg_gt, f, pickle.HIGHEST_PROTOCOL)

In [14]:
panoptic_seg_gt

{'GX010044_6.jpg': {'panoptic_mask': array([[162, 162, 162, ..., 162, 162, 162],
         [162, 162, 162, ..., 162, 162, 162],
         [162, 162, 162, ..., 162, 162, 162],
         ...,
         [162, 162, 162, ..., 108, 108, 108],
         [162, 162, 162, ..., 108, 108, 108],
         [162, 162, 162, ..., 108, 108, 108]], dtype=uint32),
  'inst2class': {2: 2,
   3: 2,
   4: 2,
   5: 2,
   6: 0,
   7: 2,
   8: 0,
   9: 2,
   10: 2,
   11: 2,
   12: 0,
   13: 0,
   14: 2,
   15: 0,
   16: 0,
   17: 0,
   18: 0,
   19: 0,
   20: 2,
   21: 0,
   22: 0,
   23: 0,
   24: 2,
   25: 2,
   26: 0,
   27: 0,
   28: 0,
   29: 0,
   30: 0,
   31: 0,
   32: 0,
   33: 0,
   34: 0,
   35: 0,
   36: 0,
   37: 0,
   38: 0,
   39: 0,
   40: 0,
   41: 0,
   42: 0,
   43: 0,
   44: 0,
   45: 0,
   46: 0,
   47: 0,
   48: 0,
   49: 0,
   50: 2,
   51: 0,
   52: 0,
   53: 0,
   54: 0,
   55: 0,
   56: 0,
   57: 0,
   58: 0,
   59: 0,
   60: 0,
   61: 0,
   62: 0,
   63: 0,
   64: 0,
   65: 0,
   66: 0,
   

In [ ]:
cp_arr = np.copy(panoptic_seg_gt["GX010042_1.jpg"]["panoptic_mask"])
cp_arr

In [ ]:
cp_arr[cp_arr==0]=999999

In [ ]:
cp_arr